<a href="https://www.kaggle.com/code/aicortex/tensorflow-vs-pytorch-regression?scriptVersionId=261506026" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# workflow setting


In [1]:
##########################
###### use in colab ######
##########################
# from google.colab import drive
# drive.mount('/content/drive')

# from pathlib import Path
# path = Path('/content/drive/MyDrive/regression-utility-script')

# %cd $path

In [2]:
###########################
###### use in kaggle ######
###########################
from pathlib import Path
path = Path('/kaggle/input/regression-utility-script')

%cd $path

/kaggle/input/regression-utility-script


# import Lib


In [3]:
import tensorflow as tf
import torch
import warnings
import sys
from importlib import import_module, reload
import numpy as np
import pandas as pd
import os
import absl.logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0=all, 1=info, 2=warning, 3=error
warnings.filterwarnings("ignore", category=RuntimeWarning)
absl.logging.set_verbosity(absl.logging.ERROR)
sys.path.append(str(path/'src/'))

2025-09-12 21:43:14.640375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757713394.820540      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757713394.874167      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def import_or_reload(module_name, path=""):
    """
    Import a module from given path.
    If already imported, reload it so changes apply.
    """
    if path not in sys.path:
        sys.path.append(path)

    if module_name in sys.modules:
        return reload(sys.modules[module_name])
    else:
        return import_module(module_name)


data_generator = import_or_reload('data_generator')
preprocess_data = import_or_reload('preprocess_data')
tf_model_regression = import_or_reload('tf_model_regression')
regression_metrics = import_or_reload('regression_metrics')
torch_model_regression = import_or_reload('torch_model_regression')

# data


## generate data


In [5]:
X_train, X_val, X_test, y_train, y_val, y_test = data_generator.regression_data()

print(X_train.shape, y_train.shape,)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(80000, 100) (80000,)
(10000, 100) (10000,)
(10000, 100) (10000,)


## preprocess data


In [6]:
out_put = preprocess_data.preprocessor(X_train, X_val, X_test,
                                       y_train, y_val, y_test)
(
    X_scaler,
    X_train_scaled,
    X_val_scaled,
    X_test_scaled,
    y_scaler,
    y_train_scaled,
    y_val_scaled,
    y_test_scaled
) = out_put

print(X_train_scaled.shape, y_train_scaled.shape,)
print(X_val_scaled.shape, y_val_scaled.shape)
print(X_test_scaled.shape, y_test_scaled.shape)

(80000, 100) (80000, 1)
(10000, 100) (10000, 1)
(10000, 100) (10000, 1)


# tensorflow model


## create and fit


In [7]:
tf_model = tf_model_regression.MyModelTF(X_scaler, y_scaler)
tf_model.fit(X_train_scaled,
             y_train_scaled,
             epochs=10,
             batch_size=32,
             validation_data=(X_val_scaled, y_val_scaled))

I0000 00:00:1757713412.807293      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/10


I0000 00:00:1757713415.339233      61 service.cc:148] XLA service 0x7d3c5c00bf90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757713415.339785      61 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1757713415.531158      61 cuda_dnn.cc:529] Loaded cuDNN version 90300


  76/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - MSE: 0.6725 - loss: 0.6725

I0000 00:00:1757713416.045597      61 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - MSE: 0.1765 - loss: 0.1765 - val_MSE: 0.0448 - val_loss: 0.0448
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - MSE: 0.0343 - loss: 0.0343 - val_MSE: 0.0274 - val_loss: 0.0274
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - MSE: 0.0254 - loss: 0.0254 - val_MSE: 0.0194 - val_loss: 0.0194
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - MSE: 0.0209 - loss: 0.0209 - val_MSE: 0.0161 - val_loss: 0.0161
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - MSE: 0.0179 - loss: 0.0179 - val_MSE: 0.0178 - val_loss: 0.0178
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - MSE: 0.0156 - loss: 0.0156 - val_MSE: 0.0134 - val_loss: 0.0134
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - MSE: 0.0138 - loss: 0.0138 - val_MSE: 0.0123 - val_loss: 0.0123
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - MSE: 0.0132 - loss: 0.0132 - val_MSE: 0.0111 - val_loss: 0.0111
Epoch 9/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/st

## evaluate


In [8]:
tf_model.evaluate(X_test_scaled, y_test_scaled)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - MSE: 0.0146 - loss: 0.0146


{'loss': 0.01472602877765894, 'compile_metrics': 0.01472602877765894}

In [9]:
tf_metrics= tf_model.my_evaluate(X_test_scaled,
                                 y_test_scaled,
                                 regression_metrics.RegressionMetrics())
tf_metrics

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


{'MIN_VALUE': -2138.485524102452,
 'MAX_VALUE': 2101.46659707183,
 'R2': 0.9996448476852704,
 'MAE': 8.287834658879245,
 'MSE': 110.56858342783922,
 'MedAE': 6.8469927615418555,
 'MAPE (unit)': nan,
 'MAPE (%)': nan,
 'SMAPE (%)': 5.679923899660318,
 'RMSLE': nan,
 'PearsonR': 0.9999138428811463}

# torch model


## create and train

In [10]:
torch_model = torch_model_regression.PyTorchRegressorWrapper(X_scaler, y_scaler)
torch_model.fit(X_train_scaled,
                y_train_scaled,
                X_val_scaled,
                y_val_scaled,
                epochs=10)

Epoch 001 | train_loss: 0.014944 | val_loss: 0.001334
Epoch 002 | train_loss: 0.001209 | val_loss: 0.001266
Epoch 003 | train_loss: 0.000780 | val_loss: 0.000510
Epoch 004 | train_loss: 0.000548 | val_loss: 0.000462
Epoch 005 | train_loss: 0.000372 | val_loss: 0.000337
Epoch 006 | train_loss: 0.000338 | val_loss: 0.000191
Epoch 007 | train_loss: 0.000262 | val_loss: 0.000251
Epoch 008 | train_loss: 0.000237 | val_loss: 0.000163
Epoch 009 | train_loss: 0.000194 | val_loss: 0.000390
Epoch 010 | train_loss: 0.000157 | val_loss: 0.000135


## evaluate model

In [11]:
torch_metrics = torch_model.my_evaluate(X_test_scaled,
                                        y_test_scaled,
                                        regression_metrics.RegressionMetrics())
torch_metrics

{'MIN_VALUE': -2138.485524102452,
 'MAX_VALUE': 2101.46659707183,
 'R2': 0.9998635081631637,
 'MAE': 5.155503398745503,
 'MSE': 42.49362434802923,
 'MedAE': 4.306975484645932,
 'MAPE (unit)': nan,
 'MAPE (%)': nan,
 'SMAPE (%)': 4.3533838221459735,
 'RMSLE': nan,
 'PearsonR': 0.9999379775919133}

# final result

In [12]:
df = pd.DataFrame([tf_metrics, torch_metrics]).T
df.columns = ['tensorflow', 'pytorch']

df

,tensorflow,pytorch
MIN_VALUE,-2138.485524,-2138.485524
MAX_VALUE,2101.466597,2101.466597
R2,0.999645,0.999864
MAE,8.287835,5.155503
MSE,110.568583,42.493624
MedAE,6.846993,4.306975
MAPE (unit),NaN,NaN
MAPE (%),NaN,NaN
SMAPE (%),5.679924,4.353384
RMSLE,NaN,NaN
